In [ ]:
from symbolic_hulls_func_aggr import *

# Geometry
from shapely.geometry import Polygon
from shapely.ops import unary_union

# Data handling and combinations
from itertools import cycle
from collections import defaultdict

# Plotly imports
import plotly.graph_objects as go


## Functions

In [ ]:
def compute_strain_energy_polynomial(elasTens, v_mol):
    """
    Compute a polynomial representation of the strain energy for 1 mole of material.

    This is achieved by multiplying the strain energy density function (defined as 
    a quadratic form involving the elastic tensor and strain vector) by the molar 
    volume of the material.

    Parameters:
        elasTens (list or array-like): A 6x6 elastic tensor in Voigt notation, representing
                                    the stiffness matrix of the material.
        v_mol (float): The molar volume of the material, in units consistent with the elastic tensor.

    Returns:
        sympy expression: A simplified polynomial expression representing the strain energy
                        as a function of the strain components (e_1, e_2, ..., e_6).
                        Numerical coefficients are rounded to the nearest integer.
    """

    e1, e2, e3, e4, e5, e6 = sp.symbols('e1 e2 e3 e4 e5 e6')
    strain = sp.Matrix([e1, e2, e3, e4, e5, e6])
    elastic_tensor = sp.Matrix(elasTens)
    strain_energy = 0.5 * v_mol * strain.T * elastic_tensor * strain

    strain_energy = sp.simplify(strain_energy)[0]

    return strain_energy.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))

## Example

In [ ]:
# a quick proof of concept

bV2O5_eTens =  [[271.8554647757902, 64.62325221396486, 27.224912079571325, -1.7191866738895836, -0.0025286851971816067, -0.002124003],
                [64.62325221396486, 197.89991427799202, 12.453223957658249, 2.119618147614496, -0.01514585462756177, -0.00177122],
                [27.224912079571325, 12.453223957658249, 33.120429472730564, 2.3253632154077684, -0.04857597532521658, -0.004906421],
                [-1.7191866738895836, 2.119618147614496, 2.3253632154077684, 33.84674175649386, 0.012452724999999758, -0.000364988],
                [-0.0025286851971816067, -0.01514585462756177, -0.04857597532521658, 0.012452724999999758, 28.376899505970716, -6.704158046],
                [-0.002124003437466659, -0.0017712195119971308, -0.0049064205224106806, -0.0003649875000002795, -6.704158045891168, 72.27092569]]

bV2O5_v_mol = 48.502

# this is the polynomial representation of the strain energy for 1 mole of bV2O5
bV2O5_energy = compute_strain_energy_polynomial(bV2O5_eTens, bV2O5_v_mol)

aV2O5_eTens =  [[266.47365146395515, 43.15159737260626, 119.5698396037316, -0.021969906572240724, -0.008038178036991589, -0.051846511],
                [43.15159737260626, 39.321322564273274, 60.251064113829564, -0.23774907491360064, 0.07308114151937194, -0.413241737],
                [119.5698396037316, 60.251064113829564, 222.77826373647477, 0.06535684464073953, -0.032256197469259985, 0.056397909],
                [-0.021969906572240724, -0.23774907491360064, 0.06535684464073953, 40.92316299266148, 0.0, -9.16E-18],
                [-0.008038178036991589, 0.07308114151937194, -0.032256197469259985, 0.0, 44.326459555780836, -1.34E-15],
                [-0.05184651084026355, -0.4132417369611278, 0.05639790945728265, -9.162304888641942e-18, -1.3365143026749432e-15, 33.76154876]]

aV2O5_v_mol = 52.872

# this is the polynomial representation of the strain energy for 1 mole of aV2O5
aV2O5_energy = compute_strain_energy_polynomial(aV2O5_eTens, aV2O5_v_mol)

e1, e2, e3, e4, e5, e6 = sp.symbols('e1 e2 e3 e4 e5 e6')
E = sp.symbols('E')
display(sp.Eq(E,bV2O5_energy), sp.Eq(E,aV2O5_energy))

In [ ]:
boundary1 = hpboundry_f2_to_f1(aV2O5_energy, bV2O5_energy)
boundary2 = hpboundry_f2_to_f1(bV2O5_energy, aV2O5_energy)
display(sp.Eq(0,boundary1), sp.Eq(0,boundary2))

## Work in progress below:

In [ ]:
def plot_high_dimensional_phase_diagram(points, labels, feature_indices = [0,1]):
    '''This will plot the phase diagram with a convex hull'''

    # Calculate the convex hull
    simplices = lower_convex_hull(points)

    # colors to cycle through
    color_cycle = cycle(['blue', 'red', 'yellow', 'green', 'purple', 'cyan', 'orange', 'pink', 'brown', 'gray'])
    
    colored_faces = defaultdict(list)

    label_color_map = {}
    for simplex in simplices:
        unique_labels = frozenset(labels[simplex])  # Using frozenset to make it hashable

        # Check if we already have a color for this unique label set
        if unique_labels not in label_color_map:
            # Assign a new color from the cycle
            label_color_map[unique_labels] = next(color_cycle)

        color = label_color_map[unique_labels]
        
        # Extract the coordinates
        x_coords = points[simplex, feature_indices[0]]
        y_coords = points[simplex, feature_indices[1]]

        # Create a 2D polygon and add it to the color group for 2D projection
        polygon = Polygon(zip(x_coords, y_coords))  # Projected onto xy-plane
        colored_faces[color].append(polygon)

    fig = go.Figure()

    # Combine polygons that share edges for each color
    for color in colored_faces:
        # Combine the list of polygons into a single geometry
        combined_polygon = unary_union(colored_faces[color])
        # Replace the list with the combined polygon
        colored_faces[color] = combined_polygon

    for color, geometry in colored_faces.items():
        # Handle both Polygon and MultiPolygon geometries
        if geometry.geom_type == 'Polygon':
            geometries = [geometry]
        elif geometry.geom_type == 'MultiPolygon':
            geometries = geometry.geoms
        else:
            continue  # Skip if not a polygonal geometry

        for poly in geometries:
            x, y = list(poly.exterior.xy[0]), list(poly.exterior.xy[1])
            fig.add_trace(go.Scatter(
                x=x,
                y=y,
                fill="toself",
                mode="none",
                fillcolor=color,
                opacity=0.5,
                name=f"2D Projection - {color}"
            ))

            # Plot interior rings (holes) if any
            for interior in poly.interiors:
                x_int, y_int = interior.xy
                fig.add_trace(go.Scatter(
                    x=x_int,
                    y=y_int,
                    fill="toself",
                    mode="none",
                    fillcolor='white',
                    opacity=1,
                    showlegend=False
                ))

    # Update layout
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X-axis', range=[-6, 6]),
            yaxis=dict(title='Y-axis', range=[-7, 5]),
        ),
        title='Phase Diagram in 2D',
        width=800,
        height=800,
        showlegend=False
    )

    fig.show()


In [ ]:
# define a nubmer of points to compute along each axis
n_points = 2

e1_vals = np.linspace(0, 1, n_points)
e2_vals = np.linspace(0, 1, n_points)
e3_vals = np.linspace(0, 1, n_points)
e4_vals = np.linspace(0, 1, n_points)
e5_vals = np.linspace(0, 1, n_points)
e6_vals = np.linspace(0, 1, n_points)

# create a meshgrid of the points
E1, E2, E3, E4, E5, E6 = np.meshgrid(e1_vals, e2_vals, e3_vals, e4_vals, e5_vals, e6_vals, indexing='ij')
E1_flat, E2_flat, E3_flat, E4_flat, E5_flat, E6_flat = E1.ravel(), E2.ravel(), E3.ravel(), E4.ravel(), E5.ravel(), E6.ravel()

# evaluate the polynomials over the mesh
bV2O5_energy_numeric = sp.lambdify((e1, e2, e3, e4, e5, e6), bV2O5_energy, 'numpy')
aV2O5_energy_numeric = sp.lambdify((e1, e2, e3, e4, e5, e6), aV2O5_energy, 'numpy')

bV2O5_energies = bV2O5_energy_numeric(E1_flat, E2_flat, E3_flat, E4_flat, E5_flat, E6_flat)
bV2O5_points = np.vstack((E1_flat, E2_flat, E3_flat, E4_flat, E5_flat, E6_flat, bV2O5_energies)).T

aV2O5_energies = aV2O5_energy_numeric(E1_flat, E2_flat, E3_flat, E4_flat, E5_flat, E6_flat)
aV2O5_points = np.vstack((E1_flat, E2_flat, E3_flat, E4_flat, E5_flat, E6_flat, aV2O5_energies)).T

# create label arrays
bV2O5_labels = np.zeros(E1_flat.shape)
aV2O5_labels = np.ones(E1_flat.shape)

In [ ]:
# combine the labels and the points
points = np.vstack((bV2O5_points, aV2O5_points))
labels = np.hstack((bV2O5_labels, aV2O5_labels))

In [ ]:
lower_convex_hull(points)

In [ ]:
plot_high_dimensional_phase_diagram(points, labels)

In [ ]:
en = np.vstack((E1_flat, E2_flat, E3_flat, E4_flat, E5_flat, E6_flat, bV2O5_energies)).T
en[1, :]

In [ ]:
# # Let's read some data in
# import json

# file_path = r'/mnt/c/Users/danie/University of Michigan Dropbox/Daniel Blevins/Research/Symbolic Hulls/ElasticTensorsV-O.json'

# with open(file_path, 'r', encoding='utf-8') as f:
#     data = json.load(f)

# phase_length = len(data)
# print(phase_length)

In [ ]:
# # print out the polymorph names in the data set
# for i in range(phase_length):
#     print(data[i]['name'])

In [ ]:
# # lets grab only the VO2 polymorphs
# VO2_data = [data[i] for i in range(phase_length) if 'VO2' in data[i]['name']]
# print(len(VO2_data))

In [ ]:
# # now grab all pairs of polymorphs by index
# import itertools

# index_list = list(itertools.combinations(list(range(0, len(VO2_data))), 2))
# index_list

In [ ]:
# # compute the tangent boundries between phase pairs, and add the points along the boundry to the total hull
# for pair in index_list:

#     # find the stress strain of the first phase
#     first_index = pair[0]
#     first_eTens = np.array(VO2_data[first_index]['elastic_tensor']['raw'])
#     first_molVol = VO2_data[first_index]['structure(relaxed)']['lattice']['volume']
#     first_phase_energy = compute_strain_energy_polynomial(first_eTens, first_molVol)
    
#     # find the stress strain of the second phase
#     second_index = pair[1]
#     second_eTens = np.array(VO2_data[second_index]['elastic_tensor']['raw'])
#     second_molVol = VO2_data[second_index]['structure(relaxed)']['lattice']['volume']
#     second_phase_energy = compute_strain_energy_polynomial(second_eTens, second_molVol)

#     # compute the boundary between the two phases
#     boundary = hpboundry_f2_to_f1(first_phase_energy, second_phase_energy)

#     # compute the points along the boundary


In [ ]:
# n = 0

# print(VO2_data[n]['name'])
# print(np.array(VO2_data[n]['elastic_tensor']['raw']))
# print(VO2_data[n]['structure(relaxed)']['lattice']['volume'])